# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
city_df = pd.read_csv("city_df.csv")
city_df.head()


,Unnamed: 0,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kapaa,22.08,-159.32,81.00,88,75,8.05,US,1603235757
1,1,Busselton,-33.65,115.33,57.00,89,0,7.18,AU,1603235948
2,2,Whitehorse,60.72,-135.05,23.00,57,75,10.29,CA,1603235823
3,3,Rikitea,-23.12,-134.97,73.80,78,0,13.60,PF,1603235949
4,4,Castro,-24.79,-50.01,65.82,83,68,3.62,BR,1603235949


In [23]:
del city_df['Unnamed: 0']
city_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,81.00,88,75,8.05,US,1603235757
1,Busselton,-33.65,115.33,57.00,89,0,7.18,AU,1603235948
2,Whitehorse,60.72,-135.05,23.00,57,75,10.29,CA,1603235823
3,Rikitea,-23.12,-134.97,73.80,78,0,13.60,PF,1603235949
4,Castro,-24.79,-50.01,65.82,83,68,3.62,BR,1603235949


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
gmaps.configure(api_key= g_key)
locations = city_df[["Lat", "Long"]]
humidity = city_df["Humidity"]
#plotting heat map
fig= gmaps.figure(center=(-10,20), zoom_level=2.3)
heat_layer = gmaps.heatmap_layer(locations, humidity, dissipating=False, max_intensity=100, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
cloudy = city_df["Cloudiness"]
windy = city_df["Wind Speed"]
narrowed_fitting = city_df.loc[(city_df["Max Temp"]> 75) & (city_df["Max Temp"]< 85)]
narrowed_fitting = narrowed_fitting.loc[cloudy == 0]
narrowed_fitting = narrowed_fitting.loc[windy< 10]
narrowed_fitting.dropna()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,Taoudenni,22.68,-3.98,81.00,18,0,9.57,ML,1603235954
81,Tessalit,20.20,1.01,82.69,15,0,8.19,ML,1603235956
156,Moroni,-11.70,43.26,77.32,71,0,2.04,KM,1603235747
194,Boueni,-12.90,45.08,75.20,83,0,2.24,YT,1603235967
234,Ondjiva,-17.07,15.73,75.04,18,0,7.23,AO,1603235971
247,Poum,-20.23,164.02,75.06,64,0,3.31,NC,1603235972
254,Dwārka,22.24,68.97,83.44,79,0,9.60,IN,1603235973
276,Bandarbeyla,9.49,50.81,78.35,73,0,8.08,SO,1603235975
357,Tobane,-21.95,27.90,77.00,18,0,6.58,BW,1603235983
373,Mitsamiouli,-11.38,43.28,78.71,72,0,2.19,KM,1603235771


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
hotel_df = narrowed_fitting
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,Taoudenni,22.68,-3.98,81.00,18,0,9.57,ML,1603235954
81,Tessalit,20.20,1.01,82.69,15,0,8.19,ML,1603235956
156,Moroni,-11.70,43.26,77.32,71,0,2.04,KM,1603235747
194,Boueni,-12.90,45.08,75.20,83,0,2.24,YT,1603235967
234,Ondjiva,-17.07,15.73,75.04,18,0,7.23,AO,1603235971
247,Poum,-20.23,164.02,75.06,64,0,3.31,NC,1603235972
254,Dwārka,22.24,68.97,83.44,79,0,9.60,IN,1603235973
276,Bandarbeyla,9.49,50.81,78.35,73,0,8.08,SO,1603235975
357,Tobane,-21.95,27.90,77.00,18,0,6.58,BW,1603235983
373,Mitsamiouli,-11.38,43.28,78.71,72,0,2.19,KM,1603235771


In [51]:
# Starting loop
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # getting lat and lon from df
    lat = row["Lat"]
    lon = row["Long"]
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # request and print url
    results = requests.get(base_url, params=params)
    hotel_address = results.json()
    # try except 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        print(f"Closest hotel is {hotel_address['results'][0]['name']}.")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel is Hotel les Arcades.
Closest hotel is Auberge bouloulou( inter djasmire ).
Closest hotel is SOS Motel.
Closest hotel is Gîte kajeon.
Closest hotel is Hotel Narayan Avezika Comfort.
Closest hotel is REER XOOSHLE GROUP.
Missing field/result... skipping.
Closest hotel is Foyer ADM.
Closest hotel is Amiin Hotel.
Closest hotel is Bahucharaji Hotel.
Closest hotel is Chez Dydyce Guesthouse.
Closest hotel is East Arnhem Real Estate.
Closest hotel is Dhukurrdji Lodge.


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
52,Taoudenni,22.68,-3.98,81.00,18,0,9.57,ML,1603235954,NaN
81,Tessalit,20.20,1.01,82.69,15,0,8.19,ML,1603235956,NaN
156,Moroni,-11.70,43.26,77.32,71,0,2.04,KM,1603235747,Hotel les Arcades
194,Boueni,-12.90,45.08,75.20,83,0,2.24,YT,1603235967,Auberge bouloulou( inter djasmire )
234,Ondjiva,-17.07,15.73,75.04,18,0,7.23,AO,1603235971,SOS Motel
247,Poum,-20.23,164.02,75.06,64,0,3.31,NC,1603235972,Gîte kajeon
254,Dwārka,22.24,68.97,83.44,79,0,9.60,IN,1603235973,Hotel Narayan Avezika Comfort
276,Bandarbeyla,9.49,50.81,78.35,73,0,8.08,SO,1603235975,REER XOOSHLE GROUP
357,Tobane,-21.95,27.90,77.00,18,0,6.58,BW,1603235983,NaN
373,Mitsamiouli,-11.38,43.28,78.71,72,0,2.19,KM,1603235771,Foyer ADM


In [52]:
hotel_name = hotel_df.dropna()
hotel_name

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
156,Moroni,-11.70,43.26,77.32,71,0,2.04,KM,1603235747,Hotel les Arcades
194,Boueni,-12.90,45.08,75.20,83,0,2.24,YT,1603235967,Auberge bouloulou( inter djasmire )
234,Ondjiva,-17.07,15.73,75.04,18,0,7.23,AO,1603235971,SOS Motel
247,Poum,-20.23,164.02,75.06,64,0,3.31,NC,1603235972,Gîte kajeon
254,Dwārka,22.24,68.97,83.44,79,0,9.60,IN,1603235973,Hotel Narayan Avezika Comfort
276,Bandarbeyla,9.49,50.81,78.35,73,0,8.08,SO,1603235975,REER XOOSHLE GROUP
373,Mitsamiouli,-11.38,43.28,78.71,72,0,2.19,KM,1603235771,Foyer ADM
374,Eyl,7.98,49.82,79.59,79,0,6.55,SO,1603235984,Amiin Hotel
376,Jodiya,22.69,70.31,84.15,74,0,8.90,IN,1603235984,Bahucharaji Hotel
433,Fayaoué,-20.65,166.53,76.15,73,0,6.82,NC,1603235990,Chez Dydyce Guesthouse


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_name.iterrows()]
locations = hotel_name[["Lat", "Long"]]

In [54]:
# Add marker layer ontop of heat map
#fig=gmaps.figure(center=(30,5), zoom_level=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig
plt.savefig("Figure ")

Figure(layout=FigureLayout(height='420px'))